In [1]:
from bs4 import BeautifulSoup
from pprint import pprint
import requests
import urllib.request


In [2]:
searchQuery = "노원 맛집"
searchOption = 'sim'  #sim or date  
page = 1
#python string template은 fstring으로 사용하자(문자열 앞에 f)
url = f'https://search.naver.com/search.naver?query={searchQuery}&sm=tab_pge&srchby=all&st={searchOption}&where=post&start={page}'

html = requests.get(url)



In [3]:
# html display, but not good using with theme(dark)
#from IPython.core.display import display, HTML
#display(HTML(html.text))

# parsing html 
soup = BeautifulSoup(html.text, 'html.parser')
# res = soup.find_all('li',{'class' : 'sh_blog_top'})
# for link in soup.find_all('a'):
# find_all(name, attrs, recursive, string, limit, **kwargs)
# find(name, attrs, recursive, string, **kwargs)
url_list = [];
for li_item in soup.find_all('li',{'class' : 'sh_blog_top'}):
    child_item = li_item.find('a',{'class' : 'sh_blog_title'})
    title = child_item.attrs['title']
    href = child_item.attrs['href']

    #url list 저장
    url_list.append(href)
    
    print(f'title : {title} , href : {href}')
    print('----------------------------------')

    

title : 기억에 남는 노원 맛집 , href : https://kyj128090.blog.me/221961099316
----------------------------------
title : 매력적인 노원 맛집 , href : https://eye4y.blog.me/221971539487
----------------------------------
title : 드디어 찾아낸 노원 맛집 , href : https://blog.naver.com/soeyoung1004?Redirect=Log&logNo=221944814459
----------------------------------
title : 갬성있던 노원 맛집 , href : https://blog.naver.com/hey_lauren?Redirect=Log&logNo=221858583217
----------------------------------
title : 완벽했던 노원 맛집 , href : https://blog.naver.com/kangmj1992?Redirect=Log&logNo=221840197703
----------------------------------
title : 노원 맛집 매콤한 쭈꾸미 , href : https://blog.naver.com/since8687?Redirect=Log&logNo=221780870365
----------------------------------
title : 전통있는 노원 맛집 , href : https://blog.naver.com/eunsun0127?Redirect=Log&logNo=221773815059
----------------------------------
title : 잘 찾아간 노원 맛집 , href : https://blog.naver.com/monkey_dream?Redirect=Log&logNo=221729194572
----------------------------------
title : 특별한맛을 

In [19]:
import os
try:
    os.mkdir('image/test')
except:
    print("error catch")

for url_item in url_list:
    # url parsing , todo : blog.me 예외처리 추가해야함
    if(url_item.find("blog.me") > 0):
        parsing = url_item;
        parsing = parsing.replace("https://","")
        nickName = parsing.split('.')[0]
        postNumber = parsing.split('/')[1]
        blogUrl = "https://m.blog.naver.com/" + nickName + "?Redirect=Log&logNo=" + postNumber
        
    else:
        blogUrl = url_item.replace("https://", "https://m.");

    print("blogUrl : " , blogUrl)

print("------ digging more -------")
#일단 테스트로 마지막 아이템 체크
blog_html = requests.get(blogUrl)
blog_soup = BeautifulSoup(blog_html.text, 'html.parser')
blog_image_class = blog_soup.find_all('div',{'class' : 'se-image'});
print("blog_image_class : " ,blog_image_class);
for index, img_item in enumerate(blog_image_class.find_all('img')):
#     print("img_item.attrs['alt'] : " , img_item.attrs['alt']);
#     if(img_item.attrs['alt'].find("프로필") > 0):
#         print("this is profile")
#         continue

    img_url = img_item.attrs['src'];
    
    if(img_url.find("w80_blur") > 0):
        img_url = img_url.replace("w80_blur" , "w800")
    print(img_url)

#     req = urllib.request.Request(imgUrl)
#     res = urllib.request.urlopen(imgUrl).read()
    
# todo--- 프로필, 지도 등 예외처리 하고 블로그별로 폴더 네이밍, 반복 할때 DDOS공격 예외처리 등등..
    urllib.request.urlretrieve(img_url, f'image/test/{index}.jpg')


error catch
blogUrl :  https://m.blog.naver.com/kyj128090?Redirect=Log&logNo=221961099316
blogUrl :  https://m.blog.naver.com/eye4y?Redirect=Log&logNo=221971539487
blogUrl :  https://m.blog.naver.com/soeyoung1004?Redirect=Log&logNo=221944814459
blogUrl :  https://m.blog.naver.com/hey_lauren?Redirect=Log&logNo=221858583217
blogUrl :  https://m.blog.naver.com/kangmj1992?Redirect=Log&logNo=221840197703
blogUrl :  https://m.blog.naver.com/since8687?Redirect=Log&logNo=221780870365
blogUrl :  https://m.blog.naver.com/eunsun0127?Redirect=Log&logNo=221773815059
blogUrl :  https://m.blog.naver.com/monkey_dream?Redirect=Log&logNo=221729194572
blogUrl :  https://m.blog.naver.com/candypowder?Redirect=Log&logNo=221776718260
blogUrl :  https://m.blog.naver.com/wlsgmldml?Redirect=Log&logNo=221799570875
------ digging more -------
blog_image_class :  [<div class="se-component se-image se-l-default" id="SE-6bb24616-6b4c-4327-9667-b7e66cdb8eab">
<div class="se-component-content se-component-content-fit"

AttributeError: ResultSet object has no attribute 'find_all'. You're probably treating a list of elements like a single element. Did you call find_all() when you meant to call find()?

In [5]:
print('Beginning file download with urllib2...')
 
url = 'https://movie.naver.com/movie/bi/mi/basic.nhn?code=157297'
req = urllib.request.Request(url)
res = urllib.request.urlopen(url).read()
 
soup = BeautifulSoup(res,'html.parser')
soup = soup.find("div",class_="poster")
#img의 경로를 받아온다
imgUrl = soup.find("img")["src"]
print(imgUrl)
#urlretrieve는 다운로드 함수
#img.alt는 이미지 대체 텍스트 == 마약왕
urllib.request.urlretrieve(imgUrl, soup.find("img")["alt"]+'.jpg')

Beginning file download with urllib2...
https://movie-phinf.pstatic.net/20181126_96/1543207321602QPWG8_JPEG/movie_image.jpg?type=m77_110_2


('마약왕.jpg', <http.client.HTTPMessage at 0x108481c50>)